In [32]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
import os
import glob
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import normalize
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense

In [2]:
path = "/content/drive/MyDrive/Games"
Path = 'C:/Users/Mullins/Desktop/Games'
gameList = glob.glob(os.path.join(Path, "*.csv"))

In [3]:
def getGame(Dataset):
    
    Data = pd.read_csv(Dataset,usecols = ['GAME_ID','HOMEDESCRIPTION','VISITORDESCRIPTION','PCTIMESTRING','PERIOD','HOME_TEAM','AWAY_TEAM','HOME_SCORE','AWAY_SCORE'])
    dfs = [df for game_id, df in Data.groupby('GAME_ID')]
    for i in range(len(dfs)):
        dfs[i]['ScoreDiff'] = dfs[i]['HOME_SCORE'] - dfs[i]['AWAY_SCORE']
        dfs[i]['PCTIMESTRING'] = dfs[i]['PCTIMESTRING'].str.split(':')
        dfs[i]['PCTIMESTRING'] = dfs[i]['PCTIMESTRING'].apply(lambda x: int(x[0]) * 60 + int(x[1]))

        dfs[i]['PCTIMESTRING'] = 720 *(dfs[i]['PERIOD'].max()-dfs[i]['PERIOD'])+dfs[i]['PCTIMESTRING']
    
        dfs[i]['NormalClock'] = (dfs[i]['PCTIMESTRING'].max() - dfs[i]['PCTIMESTRING'])
        
        dfs[i]['NormalClock'] = dfs[i]['NormalClock'].astype(float)
        HomeScore = dfs[i]['HOME_SCORE'].max()
        AwayScore = dfs[i]['AWAY_SCORE'].max()
        
        total_time = np.arange(0,dfs[i]['PCTIMESTRING'].max(),0.1)
        gameDF = pd.DataFrame(columns = ['gameTime'])
        gameDF['gameTime'] = total_time
        #If the home team outscore the away team, create an column called Outcome and add an 1 if not add an 0
        if (HomeScore > AwayScore):
            dfs[i]['OUTCOME'] = 1
        else:
            dfs[i]['OUTCOME'] = 0 
        game = gameDF.merge(dfs[i],left_on = 'gameTime',right_on = 'NormalClock', how = 'left')
        NewGameDF = game.fillna(method = 'ffill')
        NewGameDF.to_csv(f'C:/Users/Mullins/Desktop/Games/game_{i}.csv')
    return NewGameDF

def NewgetGameDF(NewGameID):
    
    NewFullGameDF1 = pd.read_csv(NewGameID)
    
    return NewFullGameDF1

def plotGameFlow(GameDF):
     
    # plot the score (home and away)
    
    #GameFlow Chart
    plt.title('Home vs away score')
    plt.ylabel('Score')
    plt.xlabel('Game Time')

    home = plt.plot(GameDF['gameTime'].to_numpy(), GameDF['HOME_SCORE'].to_numpy(),'r', label = 'home')
    away = plt.plot(GameDF['gameTime'].to_numpy(), GameDF['AWAY_SCORE'].to_numpy(),'b', label = 'away')
    plt.legend()
    plt.show()

def f(x):
    
    home_team = pd.read_csv(x)['HOME_TEAM'].iloc[0]
    away_team = pd.read_csv(x)['AWAY_TEAM'].iloc[0]
    game_name = f"{os.path.realpath(x)} ({home_team} vs {away_team})"
    
    # get the DF of interest
    gameOfInterestDF = NewgetGameDF(x)
    
    # do plotting
    plotGameFlow(gameOfInterestDF)
    
    
    
    return x


In [12]:
# interact(f, x=gameList)
interact(f,  x={game_name: f"{os.path.realpath(game)}" for game, game_name in zip(gameList,[f"({pd.read_csv(game)['HOME_TEAM'].iloc[0]} vs {pd.read_csv(game)['AWAY_TEAM'].iloc[0]})" for game in gameList])})

interactive(children=(Dropdown(description='x', options={'(Celtics vs 76ers)': 'C:\\Users\\Mullins\\Desktop\\G…

<function __main__.f(x)>

In [9]:
#How often does the home team win?
winCounter = 0

for gameID in gameList:
    BigData = NewgetGameDF(gameID)
    #Grab the final score at the end of each game
    
    
    #If the home team outscore the away team, create an column called Outcome and add an 1 if not add an 0
    
    

    winFlag = BigData['OUTCOME'].max()
        
    
    if (winFlag == 1):
        winCounter = winCounter + 1
        
print("the number of wins are %d" %(winCounter))
#print("The number of total of games" %len(NewgameList))
print("The percentage of wins are %0.2f" %(winCounter/len(gameList)))

the number of wins are 729
The percentage of wins are 0.59


In [8]:
#creates empty dataframe
x_masterTrain = pd.DataFrame()
y_masterTrain = pd.DataFrame()
x_masterTest = pd.DataFrame()
y_masterTest = pd.DataFrame()
#Splitting the game code into multiple different permutations
indices = np.random.permutation(gameList)
#Having those permutations split into a training dataset and testing data set 
trainingList, testList = indices[0:1064], indices[1065:]

for Train in trainingList:
    #grab the data
    newdfTraining = NewgetGameDF(Train)
    #take the time and place it as a value
    x_train = newdfTraining[['gameTime','ScoreDiff']]
    y_train = newdfTraining[['OUTCOME']]
    
    x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
    y_masterTrain = y_masterTrain.append(y_train,ignore_index = True)

for Test in testList:
    #grab the data
    newdfTest = NewgetGameDF(Test)
    #take the time and place it as a value
    x = newdfTest[['gameTime','ScoreDiff']]
    y = newdfTest[['OUTCOME']]
    
    x_masterTest = x_masterTest.append(x,ignore_index = True)
    y_masterTest = y_masterTest.append(y,ignore_index = True)


C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTrain = y_masterTrain.append(y_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTrain = y_masterTrain.append(y_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTrain = y_masterTrain.append(y_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTrain = y_masterTrain.append(y_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTrain = y_masterTrain.append(y_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTrain = y_masterTrain.append(y_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTrain = x_masterTrain.append(x_train,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

...

C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest = x_masterTest.append(x,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest = y_masterTest.append(y,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest = x_masterTest.append(x,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest =

C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest = x_masterTest.append(x,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest = y_masterTest.append(y,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest = x_masterTest.append(x,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest =

C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest = y_masterTest.append(y,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest = x_masterTest.append(x,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest = y_masterTest.append(y,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest =

C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest = y_masterTest.append(y,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest = x_masterTest.append(x,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest = y_masterTest.append(y,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest =

C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest = x_masterTest.append(x,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest = y_masterTest.append(y,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_masterTest = x_masterTest.append(x,ignore_index = True)
C:\Users\Mullins\AppData\Local\Temp\ipykernel_7116\911152226.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_masterTest =

In [9]:
#@title solution
model = Sequential(name="AI.Lab1-4") # Specify model type as Sequential from keras
model.add(Dense(units=12, 
                input_dim=2, 
                activation='relu', 
                name="HL1")) 
model.add(Dense(units=12, 
                activation='relu', 
                name="HL2")) 
model.add(Dense(1, 
                activation='sigmoid', 
                name="OL")) 

# Configure an optimizer
opt = tf.keras.optimizers.experimental.SGD(learning_rate=0.1, 
                                           momentum=0.9)

# Compile model
model.compile(loss='binary_crossentropy',
              optimizer = "RMSprop", # or you can use the opt optmizer from above
              metrics=['accuracy'])

# Inspect model summary
model.summary()

Model: "AI.Lab1-4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 HL1 (Dense)                 (None, 12)                36        
                                                                 
 HL2 (Dense)                 (None, 12)                156       
                                                                 
 OL (Dense)                  (None, 1)                 13        
                                                                 
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


In [10]:
estimate = model.fit(x_masterTrain.to_numpy(),
                     y_masterTrain.to_numpy(), 
                     epochs=10, # more epochs will take longer to run
                     verbose=1,
                     batch_size = 100) # change to 1 or 2 to see more details
                     #validation_split=.2, # Method 1: train/test split right here - validation = last 20% rows
                     #validation_data=[x_masterTest,y_masterTest]) # Method 2: specify validation set from sk.train_test_split here

Epoch 1/10
312802/312802 [==============================] - 273s 862us/step - loss: 0.5505 - accuracy: 0.7329
Epoch 2/10
312802/312802 [==============================] - 268s 857us/step - loss: 0.4976 - accuracy: 0.7396
Epoch 3/10
312802/312802 [==============================] - 267s 854us/step - loss: 0.4970 - accuracy: 0.7396
Epoch 4/10
312802/312802 [==============================] - 1259s 4ms/step - loss: 0.4969 - accuracy: 0.7395
Epoch 5/10
312802/312802 [==============================] - 268s 856us/step - loss: 0.4966 - accuracy: 0.7395
Epoch 6/10
312802/312802 [==============================] - 268s 855us/step - loss: 0.4965 - accuracy: 0.7396
Epoch 7/10
312802/312802 [==============================] - 296s 944us/step - loss: 0.4963 - accuracy: 0.7396
Epoch 8/10
312802/312802 [==============================] - 266s 851us/step - loss: 0.4962 - accuracy: 0.7397
Epoch 9/10
312802/312802 [==============================] - 266s 852us/step - loss: 0.4961 - accuracy: 0.7397
Epoch 10/10

In [19]:
clf = LogisticRegression(random_state=0).fit(x_masterTrain, y_masterTrain)

C:\Users\Mullins\anaconda3\envs\S52\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
# function to do something when the interactive is used
def W(game):
    
    
        
    # get the DF of interest
    gameOfInterestDF = NewgetGameDF(game)
    
    x = gameOfInterestDF[['gameTime','ScoreDiff']]
    y_AI = model.predict(x)
    # do plotting
    plotGameFlow(gameOfInterestDF)

    plt.title('AI Home Winning Probability')
    plt.ylabel('Probability')
    plt.xlabel('Game Time')
    plt.plot(x['gameTime'],y_AI)
    
    
    
    
    return game

In [33]:
widgets.interactive(W, game={game_name: f"{os.path.realpath(game)}" for game, game_name in zip(gameList,[f"({pd.read_csv(game)['HOME_TEAM'].iloc[0]} vs {pd.read_csv(game)['AWAY_TEAM'].iloc[0]})" for game in gameList])})

interactive(children=(Dropdown(description='game', options={'(Celtics vs 76ers)': 'C:\\Users\\Mullins\\Desktop…

In [30]:
def Log(game):
    
    
        
    # get the DF of interest
    gameOfInterestDF = NewgetGameDF(game)
    
    x = gameOfInterestDF[['gameTime','ScoreDiff']]
    y_pred = clf.predict_proba(x)
    # do plotting
    plotGameFlow(gameOfInterestDF)

    plt.title('Logisitic Classification Home Winning Probability')
    plt.ylabel('Probability')
    plt.xlabel('Game Time')
    plt.plot(x['gameTime'],y_pred[:,1])
    
    
    
    
    return game

In [31]:
interact(Log, game={game_name: f"{os.path.realpath(game)}" for game, game_name in zip(gameList,[f"({pd.read_csv(game)['HOME_TEAM'].iloc[0]} vs {pd.read_csv(game)['AWAY_TEAM'].iloc[0]})" for game in gameList])})

interactive(children=(Dropdown(description='game', options={'(Celtics vs 76ers)': 'C:\\Users\\Mullins\\Desktop…

<function __main__.Log(game)>